# Логистическая регрессия. Практика

В этом задании вам предлагается спрогнозировать, купит клиент велосипед или нет, обучив логистическую регрессию.

In [2]:
# подключим библиотеки
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [3]:
# считаем данные
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/bike_buyers_clean.csv')

In [3]:
# выводим первые 5 строк датафрейма
data.head(5)

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,No
1,24107,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,No
2,14177,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,No
3,24381,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Yes
4,25597,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Yes


In [4]:
# смотрим размер датафрейма
data.shape

(1000, 13)

Выведите статистики по категориальным признакам, чтобы посмотреть, сколько категорий в каждом категориальном (нечисловом) признаке.

Для этого можно воспользоваться методом `describe` из библиотеки pandas со значением параметра `include = 'object'`.

**Вопрос:** в каком категориальном признаке встречаются три различных значения?

In [4]:
objects = ['Marital Status', 'Gender', 'Education', 'Occupation', 'Home Owner', 'Commute Distance', 'Region']

In [6]:
# ваш код здесь
data.describe(include='object')

,Marital Status,Gender,Education,Occupation,Home Owner,Commute Distance,Region,Purchased Bike
count,1000,1000,1000,1000,1000,1000,1000,1000
unique,2,2,5,5,2,5,3,2
top,Married,Male,Bachelors,Professional,Yes,0-1 Miles,North America,No
freq,539,509,306,276,685,366,508,519


In [16]:
for object in objects:
    print(object, data[object].unique())

Marital Status ['Married' 'Single']
Gender ['Female' 'Male']
Education ['Bachelors' 'Partial College' 'High School' 'Partial High School'
 'Graduate Degree']
Occupation ['Skilled Manual' 'Clerical' 'Professional' 'Manual' 'Management']
Home Owner ['Yes' 'No']
Commute Distance ['0-1 Miles' '2-5 Miles' '5-10 Miles' '1-2 Miles' '10+ Miles']
Region ['Europe' 'Pacific' 'North America']


Закодируйте все категориальные столбцы с двумя категориями следующим образом:
самая часто встречающаяся категория превращается в 1, другая в 0.

In [5]:
data['Marital Status'] = data['Marital Status'].map({'Married': 1, 'Single': 0})
data['Gender'] = data['Gender'].map({'Male': 1, 'Female': 0})
data['Home Owner'] = data['Home Owner'].map({'Yes': 1, 'No': 0})

In [18]:
data.head()

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,1,0,40000,1,Bachelors,Skilled Manual,1,0,0-1 Miles,Europe,42,No
1,24107,1,1,30000,3,Partial College,Clerical,1,1,0-1 Miles,Europe,43,No
2,14177,1,1,80000,5,Partial College,Professional,0,2,2-5 Miles,Europe,60,No
3,24381,0,1,70000,0,Bachelors,Professional,1,1,5-10 Miles,Pacific,41,Yes
4,25597,0,1,30000,0,Bachelors,Clerical,0,0,0-1 Miles,Europe,36,Yes


Удалите остальные категориальные столбцы.

**Вопрос:** сколько категориальных столбцов вы удалили?

In [6]:
data = data.drop(['Education', 'Occupation', 'Commute Distance', 'Region'], axis=1)
data.head()

,ID,Marital Status,Gender,Income,Children,Home Owner,Cars,Age,Purchased Bike
0,12496,1,0,40000,1,1,0,42,No
1,24107,1,1,30000,3,1,1,43,No
2,14177,1,1,80000,5,0,2,60,No
3,24381,0,1,70000,0,1,1,41,Yes
4,25597,0,1,30000,0,0,0,36,Yes


Удалите столбец `ID`, так как он по сути нечисловой.

In [7]:
data = data.drop(['ID'], axis=1)

Сформируйте матрицу объект-признак `X` и вектор `y` с целевой переменной.
Целевая переменная - это последний столбец, `Purchased Bike`.

In [8]:
# ваш код здесь
X = data.drop(['Purchased Bike'], axis=1)
y = data['Purchased Bike']

Разбейте данные на тренировочную и тестовую часть (`Xtrain`, `Xtest`, `ytrain`, `ytest`), в тест отправьте 30% данных.
Зафиксируйте `random_state = 42`.

In [9]:
RANDOM_STATE = 42
TEST_SIZE = 0.3

In [10]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

**Вопрос:** сколько объектов в матрице `Xtrain`?

In [68]:
# ваш код здесь
Xtrain.shape

(700, 7)

Почти всё готово для обучения модели!

Осталось отмасштабировать матрицу `X`, так как линейные модели чувствительны к масштабу данных.

*  Обучите на тренировочной матрице (`Xtrain`) `MinMaxScaler` из библиотеки `sklearn.preprocessing`
*  Примените масштабирование и к `Xtrain`, и к `Xtest`
*  Переведите полученные после масштабирования `np.array` обратно в pandas `dataframe`.

Полученные масштабированные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [35]:
X.head()

,Marital Status,Gender,Income,Children,Home Owner,Cars,Age
0,1,0,40000,1,1,0,42
1,1,1,30000,3,1,1,43
2,1,1,80000,5,0,2,60
3,0,1,70000,0,1,1,41
4,0,1,30000,0,0,0,36


In [11]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(Xtrain)
Xtrain = pd.DataFrame(scaler.transform(Xtrain), columns=X.columns)
Xtest = pd.DataFrame(scaler.transform(Xtest), columns=X.columns)

Теперь обучите логистическую регрессию на тренировочных данных

In [15]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(Xtrain, ytrain)

LogisticRegression()

Сделайте предсказания на тренировочных и на тестовых данных.

In [71]:
# ваш код здесь
ypred_test = model.predict(Xtest)
ypred_train = model.predict(Xtrain)

Оцените значение accuracy на трейне и на тесте.

In [17]:
from sklearn.metrics import accuracy_score

print('test:', accuracy_score(ytest, ypred_test))
print('train:', accuracy_score(ytrain, ypred_train))

NameError: name 'ypred_test' is not defined

Качество модели получилось невысоким, зато модель не переобучена.

Попробуем добавить новых признаков в модель, используя [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

Создайте полиномиальные признаки degree = 2.

Как обычно:
*  `fit` делайте на тренировочных данных
*  `transform` и на тренировочных, и на тестовых данных. Затем верните результат к формату pandas `dataframe`.

Полученные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [73]:
from sklearn.preprocessing import PolynomialFeatures

poly_features = PolynomialFeatures(degree=2)
poly_features.fit(Xtrain)
Xtrain_ = pd.DataFrame(poly_features.transform(Xtrain))
Xtest_ = pd.DataFrame(poly_features.transform(Xtest))

**Вопрос:** на сколько признаков стало больше при добавлении полиномиальных признаков второй степени?

In [75]:
Xtrain_.shape

(700, 36)

Заново обучите логистическую регрессию, уже на расширенной матрице признаков, и сделайте предсказания на трейне и тесте, а затем оцените качество (*accuracy*).

In [76]:
# ваш код здесь
model_exp = LogisticRegression()
model.fit(Xtrain_, ytrain)
ypred_test_ = model.predict(Xtest_)
ypred_train_ = model.predict(Xtrain_)

In [77]:
print('train:', accuracy_score(ytrain, ypred_train_))
print('test:', accuracy_score(ytest, ypred_test_))

train: 0.6871428571428572
test: 0.6233333333333333


In [79]:
round(accuracy_score(ytest, ypred_test_) - accuracy_score(ytest, ypred_test), 2)

0.04

**Вопрос:** на сколько повысилось качество модели на тестовых данных?
Ответ округлите до сотых.

Появились новые требования от заказчика!

Заказчик просит, чтобы:
*  доля найденных моделью потенциальных покупателей была максимальной
*  accuracy при этом была не ниже, чем 0.6 (отклонения *accuracy* на тестовых данных на $\pm 0.05$ допустимы).

Сначала посмотрите, какие значения *recall* и *accuracy* имеют предсказния модели на тесте с классами, предсказанными по умолчанию (методом `predict`).

In [12]:
# ваш код здесь
from sklearn.metrics import recall_score

In [99]:
pd.Series(ytest).unique()

array(['Yes', 'No'], dtype=object)

In [102]:
print('acc:', accuracy_score(ytest, ypred_test_))
print('rec:', recall_score(ytest, ypred_test_, pos_label='Yes'))

acc: 0.6233333333333333
rec: 0.5526315789473685


Подберём порог для перевода вероятностей в классы, чтобы оптимизировать требуемые метрики!

Разобъем тренировочные данные на трейн и валидацию, чтобы по валидационной части подбирать порог.

In [13]:
XtrainS, Xval, ytrainS, yval = train_test_split(Xtrain, ytrain, test_size=0.3, random_state=42)

XtrainS.shape, Xval.shape

((490, 7), (210, 7))

* Обучите модель на тренировочных данных.
* Предскажите вероятности положительного класса на валидационных данных

В цикле для каждого значения порога:
*  переведите вероятности в классы
*  вычислите полноту (на валидационных данных)

Выведите на экран:

1) значение порога, дающее максимальный *recall*, при условии, что *accuracy* $\geq$ 0.6.

2) значение *recall* при этом пороге

3) значение *accuracy* для этого порога


Ищите порог на отрезке от 0 до 1 с шагом 0.01.

In [18]:
from sklearn.metrics import recall_score

RecMax = -1
BestThr = -1
BestAcc = -1
model = LogisticRegression()
model.fit(XtrainS, ytrainS)
prediction = model.predict_proba(Xval)

for thr in np.arange(0, 1, 0.01):
    classes = np.where(prediction >= thr, 'Yes', 'No')[:, 1]
    rec = recall_score(yval, classes, pos_label='Yes')
    acc = accuracy_score(yval, classes)
    if (rec > RecMax) and (acc >= 0.6):
        RecMax = rec
        BestThr = thr
        BestAcc = acc

print(BestThr, RecMax, BestAcc)

0.34 0.8529411764705882 0.6


Теперь заново обучите модель на исходных тренировочных данных (`Xtrain`, `ytrain`), предскажите вероятности на тесте и переведите их в классы по найденному порогу.

In [19]:
# ваш код здесь
model = LogisticRegression()
model.fit(Xtrain, ytrain)
pred_proba_final = model.predict_proba(Xtest)

In [20]:
prediction_final = np.where(pred_proba_final >= BestThr, 'Yes', 'No')[:, 1]
print('acc:', accuracy_score(ytest, prediction_final))
print('rec:', recall_score(ytest, prediction_final, pos_label='Yes'))

acc: 0.58
rec: 0.8552631578947368


**Вопрос:** какое значение *recall* получилось на тестовых данных после подбора порога?
Ответ округлите до десятых.

При помощи подбора порога удалось сильно увеличить значение *recall*!
Однако, как видно, на тесте не удалось сохранить условие $accuracy \geq 0.6$ (но в допустимые рамки уложились!)

Это свидетельство небольшого переобучения модели. Однако в сухом остатке имеет значительное увеличение полноты, что является приоритетом для заказчика.